In [1]:
# Libraries
# ==============================================================================
import os
os.environ["KERAS_BACKEND"] = "torch"  # 'tensorflow', 'jax´ or 'torch'
import keras

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

import skforecast
from skforecast.plot import set_dark_theme
from skforecast.datasets import fetch_dataset
from skforecast.deep_learning import ForecasterRnn
from skforecast.deep_learning.utils import create_and_compile_model
from skforecast.model_selection import TimeSeriesFold
from skforecast.model_selection import backtesting_forecaster_multiseries

from keras.optimizers import Adam
from keras.losses import MeanSquaredError
from keras.callbacks import EarlyStopping

import warnings
warnings.filterwarnings('once')

print(f"skforecast version: {skforecast.__version__}")
print(f"keras version: {keras.__version__}")

if keras.__version__ > "3.0":
    print(f"Using backend: {keras.backend.backend()}")
    if keras.backend.backend() == "tensorflow":
        import tensorflow
        print(f"tensorflow version: {tensorflow.__version__}")
    elif keras.backend.backend() == "torch":
        import torch
        print(f"torch version: {torch.__version__}")
    else:
        print("Backend not recognized. Please use 'tensorflow' or 'torch'.")

c:\Users\terre\miniconda3\envs\ff_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


skforecast version: 0.14.0
keras version: 3.8.0
Using backend: torch
torch version: 2.5.1+cpu


In [2]:
df_all = pd.read_csv(r'database\DIM_PLAYER_GAMES.csv')
df_eagles = df_all[df_all['PLAYER'].isin(['Saquon Barkley'])]

In [3]:
df_multi = pd.get_dummies(df_eagles, columns=['OPPONENT'], dtype=int)

In [4]:
df_multi.columns

Index(['GAME_ORDER', 'YEAR', 'DATE', 'GAME_NUMBER', 'WEEK', 'PLAYER', 'AGE',
       'TEAM', 'RESULT', 'RUSHING_ATT', 'RUSHING_YDS', 'RUSHING_TD',
       'RUSHING_1D', 'RUSHING_YBC', 'RUSHING_YBC_ATT', 'RUSHING_YAC',
       'RUSHING_YAC_ATT', 'RUSHING_BRKTKL', 'RUSHING_ATT_BR', 'RECEIVING_TGT',
       'RECEIVING_REC', 'RECEIVING_YDS', 'RECEIVING_TD', 'RECEIVING_1D',
       'RECEIVING_YBC', 'RECEIVING_YBC_R', 'RECEIVING_YAC', 'RECEIVING_YAC_R',
       'RECEIVING_ADOT', 'RECEIVING_BRKTKL', 'RECEIVING_REC_BR',
       'RECEIVING_DROP', 'RECEIVING_DROP_PCT', 'RECEIVING_INT',
       'RECEIVING_RAT', 'PASSING_CMP', 'PASSING_ATT', 'PASSING_CMP_PCT',
       'PASSING_YDS', 'PASSING_TD', 'PASSING_INT', 'PASSING_RATE',
       'PASSING_SK', 'PASSING_YDS1', 'PASSING_Y_A', 'PASSING_AY_A', 'PPR',
       'OPPONENT_ARI', 'OPPONENT_ATL', 'OPPONENT_BAL', 'OPPONENT_BUF',
       'OPPONENT_CAR', 'OPPONENT_CHI', 'OPPONENT_CIN', 'OPPONENT_CLE',
       'OPPONENT_DAL', 'OPPONENT_DEN', 'OPPONENT_DET', 'OPPONENT_GN

In [5]:
df_multi2 = df_multi.pivot(columns='PLAYER', values=['PPR', 'OPPONENT_ARI', 'OPPONENT_ATL', 'OPPONENT_BAL', 'OPPONENT_BUF',
       'OPPONENT_CAR', 'OPPONENT_CHI', 'OPPONENT_CIN', 'OPPONENT_CLE',
       'OPPONENT_DAL', 'OPPONENT_DEN', 'OPPONENT_DET', 'OPPONENT_GNB',
       'OPPONENT_HOU', 'OPPONENT_IND', 'OPPONENT_JAX', 'OPPONENT_LAC',
       'OPPONENT_LAR', 'OPPONENT_LVR', 'OPPONENT_MIA', 'OPPONENT_MIN',
       'OPPONENT_NOR', 'OPPONENT_NWE', 'OPPONENT_NYG', 'OPPONENT_NYJ',
       'OPPONENT_PHI', 'OPPONENT_PIT', 'OPPONENT_SEA', 'OPPONENT_SFO',
       'OPPONENT_TAM', 'OPPONENT_TEN', 'OPPONENT_WAS'], index='GAME_ORDER').reset_index().head(50)

df_multi3 = df_multi.pivot(columns='PLAYER', values=['PPR', 'OPPONENT_ARI', 'OPPONENT_ATL', 'OPPONENT_BAL', 'OPPONENT_BUF',
       'OPPONENT_CAR', 'OPPONENT_CHI', 'OPPONENT_CIN', 'OPPONENT_CLE',
       'OPPONENT_DAL', 'OPPONENT_DEN', 'OPPONENT_DET', 'OPPONENT_GNB',
       'OPPONENT_HOU', 'OPPONENT_IND', 'OPPONENT_JAX', 'OPPONENT_LAC',
       'OPPONENT_LAR', 'OPPONENT_LVR', 'OPPONENT_MIA', 'OPPONENT_MIN',
       'OPPONENT_NOR', 'OPPONENT_NWE', 'OPPONENT_NYG', 'OPPONENT_NYJ',
       'OPPONENT_PHI', 'OPPONENT_PIT', 'OPPONENT_SEA', 'OPPONENT_SFO',
       'OPPONENT_TAM', 'OPPONENT_TEN', 'OPPONENT_WAS'], index='GAME_ORDER').reset_index().head(55)

df_multi2.columns =  ['_'.join(col).strip() for col in df_multi2.columns.values]
df_multi3.columns =  ['_'.join(col).strip() for col in df_multi3.columns.values]



In [6]:
df_multi2.columns

Index(['GAME_ORDER_', 'PPR_Saquon Barkley', 'OPPONENT_ARI_Saquon Barkley',
       'OPPONENT_ATL_Saquon Barkley', 'OPPONENT_BAL_Saquon Barkley',
       'OPPONENT_BUF_Saquon Barkley', 'OPPONENT_CAR_Saquon Barkley',
       'OPPONENT_CHI_Saquon Barkley', 'OPPONENT_CIN_Saquon Barkley',
       'OPPONENT_CLE_Saquon Barkley', 'OPPONENT_DAL_Saquon Barkley',
       'OPPONENT_DEN_Saquon Barkley', 'OPPONENT_DET_Saquon Barkley',
       'OPPONENT_GNB_Saquon Barkley', 'OPPONENT_HOU_Saquon Barkley',
       'OPPONENT_IND_Saquon Barkley', 'OPPONENT_JAX_Saquon Barkley',
       'OPPONENT_LAC_Saquon Barkley', 'OPPONENT_LAR_Saquon Barkley',
       'OPPONENT_LVR_Saquon Barkley', 'OPPONENT_MIA_Saquon Barkley',
       'OPPONENT_MIN_Saquon Barkley', 'OPPONENT_NOR_Saquon Barkley',
       'OPPONENT_NWE_Saquon Barkley', 'OPPONENT_NYG_Saquon Barkley',
       'OPPONENT_NYJ_Saquon Barkley', 'OPPONENT_PHI_Saquon Barkley',
       'OPPONENT_PIT_Saquon Barkley', 'OPPONENT_SEA_Saquon Barkley',
       'OPPONENT_SFO_Saquon 

In [24]:
# Model creation
# ==============================================================================
series = ['PPR_Saquon Barkley', 'OPPONENT_ARI_Saquon Barkley',
       'OPPONENT_ATL_Saquon Barkley', 'OPPONENT_BAL_Saquon Barkley',
       'OPPONENT_BUF_Saquon Barkley', 'OPPONENT_CAR_Saquon Barkley',
       'OPPONENT_CHI_Saquon Barkley', 'OPPONENT_CIN_Saquon Barkley',
       'OPPONENT_CLE_Saquon Barkley', 'OPPONENT_DAL_Saquon Barkley',
       'OPPONENT_DEN_Saquon Barkley', 'OPPONENT_DET_Saquon Barkley',
       'OPPONENT_GNB_Saquon Barkley', 'OPPONENT_HOU_Saquon Barkley',
       'OPPONENT_IND_Saquon Barkley', 'OPPONENT_JAX_Saquon Barkley',
       'OPPONENT_LAC_Saquon Barkley', 'OPPONENT_LAR_Saquon Barkley',
       'OPPONENT_LVR_Saquon Barkley', 'OPPONENT_MIA_Saquon Barkley',
       'OPPONENT_MIN_Saquon Barkley', 'OPPONENT_NOR_Saquon Barkley',
       'OPPONENT_NWE_Saquon Barkley', 'OPPONENT_NYG_Saquon Barkley',
       'OPPONENT_NYJ_Saquon Barkley', 'OPPONENT_PHI_Saquon Barkley',
       'OPPONENT_PIT_Saquon Barkley', 'OPPONENT_SEA_Saquon Barkley',
       'OPPONENT_SFO_Saquon Barkley', 'OPPONENT_TAM_Saquon Barkley',
       'OPPONENT_TEN_Saquon Barkley', 'OPPONENT_WAS_Saquon Barkley']
levels = ['PPR_Saquon Barkley']   # Features to predict. It can be all the series or less
lags = 12
steps = 5 
data = df_multi2[series].copy()
data_train = df_multi2[series].copy()
data_val = df_multi2[series].copy()
data_test = df_multi2[series].copy()

model = create_and_compile_model(
    series=data_train,
    levels=levels, 
    lags=lags,
    steps=steps,
    recurrent_layer="LSTM",
    recurrent_units=[100, 50],
    dense_units=[64, 32],
    optimizer=Adam(learning_rate=0.01), 
    loss=MeanSquaredError()
)

forecaster = ForecasterRnn(
    regressor=model,
    levels=levels,
    steps=steps,
    lags=lags,
)

keras version: 3.8.0
Using backend: torch
torch version: 2.5.1+cpu


In [25]:
forecaster.fit(data_train)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.1626


In [26]:
predictions = forecaster.predict()
predictions

,PPR_Saquon Barkley
50,13.634933
51,2.401607
52,13.349408
53,24.401941
54,14.836530


In [27]:
df_multi3[['PPR_Saquon Barkley']].tail(5)

,PPR_Saquon Barkley
50,17.5
51,15.2
52,23.0
53,5.5
54,15.2
